In [37]:
import sqlite3

In [38]:
conn = sqlite3.connect("students.db")
cursor = conn.cursor()

In [39]:
create_table_query = """
create table STUDENT(NAME VARCHAR(50), CLASS VARCHAR(10), GRADE VARCHAR(20))
"""

cursor.execute(create_table_query)

In [40]:
cursor.execute("INSERT INTO STUDENT VALUES('Anirudh', 'ML', 'A+')")
cursor.execute("INSERT INTO STUDENT VALUES('Ram', 'DS', 'A+')")
cursor.execute("INSERT INTO STUDENT VALUES('Krishna Saha', 'ML', 'B+')")

In [41]:
res = cursor.execute("SELECT * FROM STUDENT")
for record in res:
    print(record)

conn.commit()
conn.close()

('Anirudh', 'ML', 'A+')
('Ram', 'DS', 'A+')
('Krishna Saha', 'ML', 'B+')


In [42]:
from langchain.agents import create_sql_agent
from langchain.sql_database import SQLDatabase

In [43]:
from typing_extensions import TypedDict

class AppState(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [44]:
from langchain_ollama.chat_models import ChatOllama

llm = ChatOllama(model="mistral:latest")

In [45]:
from langchain_core.prompts import ChatPromptTemplate

system_message = """
Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}
"""

user_prompt = "Question: {input}"

query_prompt_template = ChatPromptTemplate([
    ("system", system_message),
    ("user", user_prompt)
])

for message in query_prompt_template.messages:
    message.pretty_print()

================================ System Message ================================


Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}

================================ Human Message =================================

Question: {input}


In [46]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///students.db")
print(db.dialect)

sqlite


In [47]:
db.run("SELECT * FROM STUDENT;")

"[('Anirudh', 'ML', 'A+'), ('Ram', 'DS', 'A+'), ('Krishna Saha', 'ML', 'B+')]"

In [48]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]

def write_query(state: AppState):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [49]:
write_query({"question": "How many total students are there?"})

{'query': 'SELECT COUNT(*) FROM STUDENT'}

In [50]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: AppState):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

In [51]:
execute_query({'query': 'SELECT COUNT(*) FROM STUDENT'})

{'result': '[(3,)]'}

In [52]:
def generate_answer(state: AppState):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f"Question: {state['question']}\n"
        f"SQL Query: {state['query']}\n"
        f"SQL Result: {state['result']}"
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

In [53]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(AppState).add_sequence(
    [write_query, execute_query, generate_answer]
)
graph_builder.add_edge(START, "write_query")
graph = graph_builder.compile()

In [54]:
for step in graph.stream(
    {"question": "How many students are there?"}, stream_mode="updates"
):
    print(step)

{'write_query': {'query': 'SELECT COUNT(*) FROM STUDENT;'}}
{'execute_query': {'result': '[(3,)]'}}
{'generate_answer': {'answer': " Based on the provided SQL query and result, there are 3 students in the database table 'STUDENT'."}}


In [55]:
for step in graph.stream(
    {"question": "Can you add a student named Rajesh from CSE with grade of A+?"}, stream_mode="updates"
):
    print(step)

{'write_query': {'query': "INSERT INTO STUDENT (NAME, CLASS, GRADE) VALUES ('Rajesh', 'CSE', 'A+')"}}
{'execute_query': {'result': ''}}
{'generate_answer': {'answer': ' Based on the SQL query and result provided, a new record has been successfully added to the "STUDENT" table with the name "Rajesh", majoring in "CSE", and having a grade of "A+". So, yes, the student named Rajesh from CSE has been added with a grade of A+.'}}


In [56]:
for step in graph.stream(
    {"question": "Can you give me details about Anirudh"}, stream_mode="updates"
):
    print(step)

{'write_query': {'query': "SELECT NAME, CLASS, GRADE FROM STUDENT WHERE NAME = 'Anirudh' LIMIT 1"}}
{'execute_query': {'result': "[('Anirudh', 'ML', 'A+')]"}}
{'generate_answer': {'answer': ' Based on the provided SQL query and result, Anirudh is a student in Class ML with a grade of A+.'}}
